In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import snorkel

%matplotlib inline
from IPython.core.pylabtools import figsize

In [2]:
from snorkel.labeling.apply import PandasLFApplier
from snorkel.labeling.lf import labeling_function

POS = 1
NEG = -1
ABSTAIN = 0

In [3]:
#Load in our cleaned data and re-check category labels
media_df = pd.read_csv('/Users/awhite/Documents/snorkel/arabic_news_cleaned.csv')
media_df.category.value_counts()

كرة_القدم                       4301
الأزمة_السورية                 3160
جماعات_مسلحة                    2142
أسواق_النفط                    1364
لاجئون                         1256
رياضات_اخرى                     1100
المعارضة_السورية                1002
صواريخ                           741
التقنية_والمعلومات               639
فضاء                             630
أسلحة_ومعدات_عسكرية             625
الهجرة_إلى_أوروبا              616
الأزمة_الأوكرانية              595
اكتشافات                         577
الأزمة_اليمنية                  566
تفجيرات                          559
مشاهير                           539
البحوث_الطبية                    530
معلومات_عامة                     484
طائرات_حربية                    479
الانتخابات_الأمريكية            447
جرائم                           433
مظاهرات                          422
مؤشرات_اقتصادية                 400
عقوبات_اقتصادية                  397
الاعتراف_بدولة_فلسطين            369
امراض                            323
أولمبياد_ريو_د

There are several broad categories like جماعات_مسلحة (armed groups) and أسواق_النفط (oil markets), then more specific categories such as two for the Syrian conflict: الأزمة_السورية (Syrian conflict) and المعارضة_السورية (Syrian opposition). Let's start with one of the general categories, then move on to Syria.  

In [20]:
#Recoding data for these two classification problems
#Full category names not working, probably because Arabic

media_df = media_df.assign(oil = media_df.category.str.contains("النفط") == True)
media_df = media_df.assign(syria = media_df.category.str.contains("سورية") == True)

media_df.oil = media_df.oil.replace({True:1,False:-1})
media_df.syria = media_df.syria.replace({True:1,False:-1})

media_df[media_df.oil == 1].head()

,text,category,oil,syria
6208,اسعار تتراجع توقعات انتاج اوبك مستوياته حاليه هبطت اسعار بفعل صعود دولار توقعات تبقي منظمه اوبك انتا مرتفعا مخاوف استمرار معروض اسواق اظهرت نتائج انتاج منظمه اوبك اعلى مستوى عامين مليون برميل يوميا ماضي تجتمع منظمه اوبك فيينا مقبل يتوقع تغير سياس انتاجيه سيقود لاستمرار تخمه معروض اسواق عالميه مستقبل منظور بحلول بتوقيت موسكو انخفضت عقود آجله لسعر مزيج برنت تسليم بنسبه يعادل دولار دولار برميل تراجعت عقود آجله امريكي تسليم بنسبه يعادل سنتا دولارا برميل ارتفع موشر دولار بنسبه مقابل عملات يجعل تكلفه حائزي عملات يشار اسعار قفزت بنحو ماضي اكبر ارتفاع خلفيه تراجع عمليات تنقيب ولايات متحده,أسواق_النفط,1,-1
6209,تبدا ببنا انابيب سيبيريا روسي اراض قالت شركه غازبروم روسيه ببنا انابيب لنقل اراض سيكون امتدادا انابيب سيبيريا روسي لنقل طبيعي روسيا مفترض خطين صيني روسي اواخر جاري اكدت غازبروم قابضه روسي انابيب سيبيريا مخصص لنقل طبيعي شرقي يجري وفقا جدول زمني محدد سابقا يفترض انبوب سيبيريا اراضي صينيه نهايه رئيس مجلس اداره غازبروم يكسي ميلر بكين نائب لرئيس مجلس دوله صيني تشانغ رئيس مجلس اداره شركه وطنيه صينيه لمناقشه مفاوضات بشان امدادات روسي غربي تحديد جدول زمني استعداد لتوقيع احمر خارطه يشير مسار انبوب سيبيريا لنقل طبيعي اراضي غازبروم نهايه شركه وطنيه صينيه عقدا لتوريد روسي شرقي لمده عاما توريد مليار مكعب سنويا مفترض تغذيه شرقي طبيعي روسي حقول انتاج مقاطعتي ياكوتسك اركوتسك روسيتين انابيب رئيسي سيبيريا اوائل ماضي شركه غازبروم شركه وطنيه صينيه اتفاقا بشان شروط اساسيه لتوريد طبيعي روسيا غربي يفترض تغذيته حقول طبيعي روسيه سيبيريا باستطاعه تبلغ مليار مكعب سنويا نوفوستي,أسواق_النفط,1,-1
6210,اسعار ترتفع بدعم ارتفعت اسعار بدعم نزوله تعاملات مبكره بفعل توقعات اوبك تخفض انتا اجتما اسبوع سعودي استراتيجيه اوبك ناجحه مقرر تجتمع منظمه بلدان بترول اوبك باكثر انتاج عالم مقبل فيينا مصرف امريكي تبقي منظمه انتا قالت شركه استشارات ستجتمع اوبك تميل انتاج اسعار تلقت دعما بترول سعودي نعيمي سيرتفع مصرف مصافي تدعم اسعار بحلول ساعه بتوقيت موسكو مزيج برنت عقود اقرب استحقاق سنتا دولار برميل امريكي سنتا دولار برميل,أسواق_النفط,1,-1
6211,اوبك تجتمع فيينا توقعات ابقا مستويات انتاج تجتمع اعضا منظمه اوبك فيينا توقعات حفاظ مستويات انتاج حاليه طهران ايجابيه مفاوضات نووي مندوب خليجي رفيع منظمه اوبك اعضا خليجيين رئيسيين اوبك يجمعون ابقا انتاج تغيير معززا تسود متزايد اعضا مندوب اجتماع رسمي اعضا خليجيين اربعه رئيسيين اوبك كويت سعوديه امارات بلدان خليج اعضا اوبك اجماع ابقا انتاج دونما تغيير مضيفا يريد قارب متوقع يسير اجتماع بسلاسه سعودي وزير طاقه قطري امين لمنظمه اوبك ندوه قبيل اجتماع اوبك اربع حوالي مليون برميل يوميا يوازي انتاج اوبك بالغ مليون برميل تنتج سعوديه حدها مليون برميل اعضا اخرون فنزويلا ايران تقليص انتاج لدعم اسعار خسرت تقريبا برميل برنت مرجعي تقريبا مستوى دولارا يبلغ دولارا برميل امريكي خفيف مستوى دولارات يبلغ دولارا اسعار خسرت تقريبا تنوي اوبك انتا امين لمنظمه اوبك بدري منظمه تستهدف احدا بعينه منتجي استراتيجي محافظه سوقيه سياق سعودي نعيمي توقعات طاقه طويل تبدو ايجابيه غايه موكدا منظمه تلبي عالمي مستبعدا يتغير جهته توقع قطري محمد ساده توازنا مشيرا عددا اسباب تبعث تفاول بشان مستقبلا لافتا تعافي اقتصادي عالمي يظهر موشرات مشجعه يتحسن,أسواق_النفط,1,-1
6212,روسيا تنوي محافظه مستويات انتاج حاليه طاقه روسي كسندر نوفاك بلاده تهدف طويل محافظه مستويات انتاج حاليه حوالي مليون برميل يوميا اشار مسوول روسي ندوه فيينا اوبك مسوولين تنفيذيين لشركات عالميه كبرى احتياطات روسيه تعتبر ارخص عالميا كلفه استخراج تتوقع روسيا تتراوح اسعار دولارا برميل غضون سنوات ثلاث مقبله نوفاك اسعار تبعث ارتياح لصناعه عالميه توقع نوفاك تشهد سنوات قليله قادمه نقصا معروض عالم تجميد مشاريع نتيجه انخفاض اسعار اويل تناقش عراق انشا شركات مشتركه اكتشاف جيولوجي لتوسيع مشروع قرنه صعيد متصل كبيروف رئيس شركه اويل ثاني اكبر شركه نفطيه روسيه ختام لقائه عراقي عادل مهدي هامش اجتماعات اوبك فيينا شركته تناقش عراق انشا شركات مشتركه اكتشاف جيولوجي لتوسيع مساحه آبار استخراج قرنه تعمل اويل تطويره جهته اعتبر عراقي مشاريع مشتركه شركه اويل روسيه اولويات عراق مهدي شركه اويل تعتبر شريكا هاما عراق مشيرا عقود جيده تعمل تعمل مستقبل جدير شركه اويل تعمل حاليا تطوير قرنه عراق,أسواق_النفط,1,-1


In [21]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(media_df, test_size = 0.2, random_state = 0)

train, valid = train_test_split(train, test_size = 0.2, random_state = 0)
train, dev = train_test_split(train, test_size = 0.2, random_state = 0)

Y_train = train["oil"].values
Y_dev = dev["oil"].values
Y_valid = valid["oil"].values
Y_test = test["oil"].values

len(train)

14166

In [22]:
pd.set_option("display.max_colwidth", 0)

dev[dev.oil == 1].sample(5)

,text,category,oil,syria
6441,ايران تستعد لاستئناف شحنات اوروبا تخطط طهران لشحن برميل يوميا اوروبيه اصبحت شركه جلينكور شركه غربيه تتعامل ايراني اظهرت بيانات لتتبع ناقله استاجر جلينكور شركه انجلو سويسريه متخصصه تجاره اوليه تحمي بنحو ايراني ماضي موقع معلومات زاره ايرانيه انترنت شانا ايراني بيجن زنغنه قوله مبيعات ايراني لاوروبا عقوبات دوليه طهران تجاوزت برميل اضاف وزير ايراني طهران تخطط لزياده صادرات بمقدار برميل يوميا اشهر قادمه صادرات ايرانيه ستتجه اسواق اولى يكشف ايراني احجام ايراني متجهه اوروبا موقع زنغنه قوله شركه توتال فرنسيه ستشتري برميل ايران لمسات اخيره ستوضع جاري شركه ايني ايطاليه مهتمه بشرا برميل ايران مضيفا ممثلي شركه ايطاليه سيزورون طهران مستقبل قريب لمناقشه شروط زنغنه شركه ساراس ايطاليه لتكرير مهتمه بشرا برميل ايران تامل طهران استثمارات اجنبيه بنحو مليار دولار لاستثما صناعه ايرانيه استكشاف تنميه,أسواق_النفط,1,-1
6311,يصعد تنامي مخاوف بشان ارتفعت اسعار تداولات مستثمرين تاثير تباطو اقتصاد صيني عالمي بحلول ساعه بتوقيت موسكو ارتفع مزيج برنت عقود آجله تسليم بمقدار سنتا نسبته دولار برميل صعدت عقود آجله امريكي تسليم بمقدار سنتا بنسبه دولار برميل قالت مجموعه باركليز خدمات بنكيه مذكره تحليليه نعتقد اسعار مواد اوليه نحاس تقترب معدلا عاديه محدود منافسه منتجين تشكل ضغطا يوثر تباطو اقتصادي مستهلك طاقه عالم يساهم انخفاض اسعار اشارت بيانات اقتصاديه نشرت ارباح صناعيه ثاني اكبر اقتصاد عالم تراجعت ماضي بنسبه مقارنه نفسه ماضي هبطت ارباح قطاع صناعه بنسبه صعيد متصل توقع مدير تنفيذي لوكاله طاقه دوليه فاتح بيرول يبقى مستوى دولارا برميل لفتره طويله سوال لصحيفه نمساويه اجرت مقابله مدير تنفيذي احتمال تتجاوز اسعار مستوى دولارا برميل مجددا بيرول يمكنني سيظل منخفضا لبضع فصول,أسواق_النفط,1,-1
21392,روسيا تجذب مستثمرين وول ستريت عقد زير طاقه روسي لقا مستثمرين امريكيين هامش مشاركته موتمر طاقه بهيوستن وصف لقا ايجابي موكدا اوساط امريكيه ابدت اهتماما بمشاريع روسيا اعقاب لقائه مستثمرين كسندر نوفاك لقا مغلقا بشكل بوسعي اقول ايجابيا عمليا جدا مشيرا مستثمرين اجانب معرفه جيده بقطاع نفط غاز روسي وضع اقتصاد اتحاد روسي يكشف وزير روسي مزيد تفاصيل اجتماع ممثلين مختلف شركات صناديق استثماريه عالم ذكرت متحدثه باسم زاره طاقه روسيه وقت لاحق اجتماع ممثلين شركه فيديليتي استثمارات مجموعه اوخ زيف استثمار جهاز ابوظبي استثمار صندوق طريق حرير شركه صين استثمار غيرهم نوفوستي فريد غايرلي,أسواق_النفط,1,-1
21016,اسعار نفط تتعافى هبوط حاد تعافت اسعار نفط بنحو طفيف اَب منيت بخسائر فادحه جلسه سابقه بقيت ادنى مستويا اعوام نصف استمرار ضعف اسهم صينيه بحلول ساعه بتوقيت موسكو ارتفع سعر مزيج برنت عقود آجله تسليم شهر بمقدار سنتا نسبته دولار برميل صعد سعر خام امريكي عقود آجله تسليم شهر بمقدار سنتا نسبته دولار برميل خامان قرب ادنى مستوياتهما سجلا جلسه سابقه دولار خام امريكي دولار لخام برنت يشير مخاوف بشان آفاق اقتصاديه صين تتزايد وقت تشهد اسواق عالميه تخمه معروض يرى خبرا مستثمرين يشعرون خوف ناجم هبوط سوق اسهم صينيه انهيار اسعار مواد اوليه خطر انكماش تباطو اقتصادي ثاني اكبر اقتصاد عالم اكبر دوله مستهلكه سلع اوليه دقت اجراس انذار اسواق عالميه انهت بورصه شنغهاي تعاملات ادنى مستويا صدور بيانات اقتصاديه صينيه ضعيفه خوفا مستثمرين يزال انتاج نفط كبيرا جدا بلدان منظمه دول نفط اوبك يتجاوز سقف انتا محدد مليون برميل يوميا اوضاع يودي تراجع طلب نفط صين اول مستورد طاقه عالم سوى زياده فائض عرض تخشى اسواق عوده نفط ايراني اتفاق ابرم برنامج نووي لطهران تخطط ايران عضو اوبك زياده انتا نفط خام فور رفع عقوبات غربيه بمقدار الف برميل يوميا بواقع مليون برميل غضون شهور,أسواق_النفط,1,-1
6992,اسعار تقفز تفاول كاله طاقه دوليه قفزت اسعار تداولات بيانات كاله طاقه دوليه مساهمه انتاج برفع اسعار بحلول ساعه بتوقيت موسكو بتوقيت غرينيتش ارتفع برميل عالمي مزيج برنت بنسبه بمقدار سنتا ليصل دولار برميل امريكي خفيف بنسبه بمقدار سنتا ليبلغ دولار قالت وكاله تقري شهري يتطلعون استعاده تواز رساله موضحه اصلت اوبك التزام باتفاق انتاج تشهد عجزا قدره برميل يوميا سيدفع اسعار صعود ناديجدا انيوتينا,أسواق_النفط,1,-1


In [23]:
#testing regex quirks in Arabic text
test = "الجزائر"
test2 = "الجزاىر"

test.count(r"الجزا*ر")

#Wildcard not working

0

In [24]:
oil = r"برميل|نفظ|بترول"

@labeling_function()
def oil_terms(x):
    return POS if re.search(oil, x.text) else ABSTAIN

In [25]:
econ = r"سعر|اسعار|ستثمار|اقتصاد|سوق|اسواق"

@labeling_function()
def econ_terms(x):
    return POS if re.search(econ, x.text) else ABSTAIN

In [26]:
conflict = r"ضبرات|معارك|سلح"

@labeling_function()
def conflict_terms(x):
    return NEG if re.search(conflict, x.text) else ABSTAIN

In [27]:
exclude = r"كرة القدم|فلم|افلام"

@labeling_function()
def misc_exclude(x):
    return NEG if re.search(exclude, x.text) else ABSTAIN

In [28]:
countries = r"خليجي|الجزائر|ليبي|نيجيري|سعودي|روسي|امريك"

@labeling_function()
def oil_countries(x):
    return POS if re.search(countries, x.text) else ABSTAIN


In [29]:
lfs = [oil_terms, econ_terms, conflict_terms, misc_exclude, oil_countries]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=train)
L_dev = applier.apply(df=dev)


100%|██████████| 3542/3542 [00:01<00:00, 2706.50it/s]

In [30]:
from snorkel.labeling.analysis import LFAnalysis

LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
oil_terms,0,"[0, 1]",1.000000,1.000000,0.618577,104,73,0.029362
econ_terms,1,"[0, 1]",1.000000,1.000000,0.618577,143,34,0.040373
conflict_terms,2,[0],0.792772,0.792772,0.484190,0,168,0.000000
misc_exclude,3,[0],0.992660,0.992660,0.614060,0,177,0.000000
oil_countries,4,"[0, 1]",1.000000,1.000000,0.618577,151,26,0.042631


In [32]:
from snorkel.analysis.error_analysis import get_label_buckets

buckets = get_label_buckets(Y_dev, L_dev[:, 1])
dev.iloc[buckets[(POS, ABSTAIN)]].sample(5, random_state=1)

,text,category,oil,syria
21065,غارديان تتحصل ثائق داعش لبنا دوله خلافه نشرت صحيفه غارديان بريطانيه ثيقه سريه مسربه تعود لداعش تكشف تفاصيل خطط تنظيم ارهابي لبنا اسس يدعي دوله خلافه مرحله اولى اعلان تنظيم خلافه اسلاميه اراضي عراق سوريا ركز قادته اصدار اوامر تخص جوانب سطحيه حياه اجتماعيه حدود دوله مزعومه حظر ارتدا ملابس اسلاميه تربيه حمام اعتبار اخيره مضيعه وقت انتقل داعش اصدار لوائح تعليمات تخص دوله بحد نشرت غارديان ثيقه سريه مكونه صفحه تحمل عنوان مبادئ اداره دوله اسلاميه تضم وثيقه مذكوره مبادئ نظام تربوي اعلامي لداعش آليات دعايته علاقاته خارجيه تحكم تجاره نفط غاز تصور وثيقه نظاما شديد تعقيد لتسيير معسكرات تنقسم اعداديه خاصه اطفال تدريبيه لمسلحين محترفين تشير صحيفه برنامج يبرز مدى جديه تعاطي تنظيم ارهابي قضيه اسس دوله غارديان تحدي يواجهه ائتلاف غربي داعش ينحصر تنفيذ مهمه قتاليه عاديه تنظيم اعتباره مجرد جمله مسلحين تنقل صحيفه ستينلي ماكريستال جنرالات احتياط امريكيين قوله غرب بتعامله داعش عصابه مجانين عاديين يجازف وقوع خطر تقدير قواه صحيفه بريطانيه تشير ثائق داعش صادره مدار اشهر خمسه اخيره تركز اجراات امن تعبئه تجنيد امر يدل تنامي اعراض بارانويا صفوف تنظيم فرض حظرا شبكات الـ خاصه اعلن عاما هاربين ساحات قتال نقص جنود تنظيم نوفوستي,أسواق_النفط,1,-1
21414,مصر ثالث كشف غاز لشركه اعلنت شركه نفط بريطانيه اكتشافا جديدا غاز طبيعي منطقه امتياز شمال دمياط بحريه بشرق دلتا نيل يوكد جود مكامن لغاز منطقه مصريه يعد ثالث نجاح لنشاطـ قطاع كشفي سلامات اتول وتم حفر بئر استكشافيه اطلق اسم قطاميه ضحله بعمق اجمالى مترا مياه امتار تقريبا تدل معطيات اوليه جود طبقه حامله غاز بسماكه مترا تجري حاليا دراسه خيارات متعلقه بربط كشف بنيه تحتيه قائمه بوب دودلي رئيس تنفيذى لمجموعه عالميه يعتبر بئر قطاميه كشف منطقه امتياز نقوم فعل بتطوير حقل اتول تقييم اكتشاف سلامات نجاحنا مستمر مجال يوكد ايماننا دلتا نيل حوض غاز طراز عالمي تقع بئر قطاميه ضحله شمال مدينه دمياط تبعد جنوب غرب حقل سلامات غرب تسهيلات حقل حابى بحريه يذكر شركه تنتج نحو اجمالي غاز مصري فريد غايرلي,أسواق_النفط,1,-1
21057,موسكو تعلق توريدات روسي اوكرانيا اعلن يكسي ميللر رئيس شركه غازبروم روسيه تعليق امدادات طبيعي روسي اوكرانيا استلام دفعات نقديه جديده كييف توريدات روسيا تعتزم ايقاف ارداتها لاوكرانيا تنظر تعليق امدادات رئيس شركه غازبروم روسيه شركته تتلق دفعات جديه شركه طاقه اوكرانيه نفطوغاز توريدات جديده تعليق امدادات بحلول ساعه بتوقيت موسكو روسيا اوكرانيا مفوضيه اوروبيه نهايه ماضي بروتوكول مبدئي بشان توريدات روسي اوكرانيا يعرف حزمه شتويه اتفاق قيام كييف بدفع مسبقا روسي مورد تضمن اتفاق كييف تخفيضات توفير تمويل لازم لاوكرانيا مفوضيه اوروبيه لشرا كميات مطلوبه طبيعي لازم مستودعات اوكرانيه بهدف ضمان امدادات مستقره روسي اوروبا اراضي اوكرانيه,أسواق_النفط,1,-1
21133,شركه بريطانيه تسحب موظف جزائر كشفت شركه بريتش بتروليوم ستسحب موظف محطتي عين صالح ميناس غاز جزائر اسبوعين مقبلين اعلنت شركه نفط غاز نرويجيه شتات اويل تشغل منشاتين شراكه خطط لتخفيض موظف هجوم قذائف صاروخيه منشاه عين صالح يسفر قوع اصابات خسائر قالت شركه بريطانيه بيان قررت اجرا نقل موقت مراحل لجميع افراد طاق مشروعي عين صالح اميناس جزائر اسبوعين مقبلين اتخذ قرار كاجرا احترازي وقت ماضي استهدف مسلحون مجهولون قاعده نفطيه بعين صالح جزائر صواريخ انبا قوع ضحايا نقلت صحيفه خبر جزائريه مصادر امنيه عناصر جيش وطني شعبي تصدت لمحاوله اعتدا ارهابي منشاه نفطيه اقعه منطقه خريشبه كلم عين صالح اضافت جماعه ارهابيه استهدفت قاعده نفطيه تابعه شركه بريطانيه بريتيش بتروليوم قذائف تمهيدا استيلا تدخل عناصر قوات جيش وطني شعبي احبط محاوله,أسواق_النفط,1,-1
6757,تشغيل آبار طبيعي جديده متوسط كشفت زاره بترول مصريه نجاح ايني ايطاليه بريتش بتروليوم بتشغيل آبار جديده طبيعي متوسط اكدت وزاره مليون مكعب يوميا انتاج بنهايه ماضي بفضل تمكن شركات اجنبيه مذكوره تشغيل آبار اربعه متوسط دلتا صحرا غربيه افاد بيان وزاره آبار اربعه بمنطقه شمال دلتا تابع لشركه بمعدل انتاج مليون مكعب يوميا بمعدل انتاج مليون مكعب يوميا علما بئرين تابعين لحقل نورس تشرف ساهم بزياده انتاج ليصل مليون مكعب يوميا بمعدل انتاج مليون مكعب يوميا رابع سترا صحرا غربيه بمعدل انتاج مليون مكعب يوميا يذكر متوسط انتاج يبلغ حوالي مليار مكعب يوميا تستهلك بلاد حوالي مليار مكعب تعوض استيراد تراجع انتاج زياده استهلاك ايام دوله مصدره طاقه تحويل امدادات طاقه محليه مستورده طاقه باستيراد طبيعي مسال,أسواق_النفط,1,-1


Hm...a few entries here are definitely not about the oil market. Bad labeled data! What an unusual problem!

Other errors seem to indicate we can just expand the keywords in existing LFs and get better results.

In [46]:
oil = r"غاز|برميل|نفظ|بترول"
econ = r"سعر|اسعار|ستثمار|اقتصاد|سوق|اسواق|شركة"
conflict = r"ضربات|اهلي|عرقي|معارك|سلح"
exclude = r"اسلام|كرة القدم|فلم|افلام|ثقافي"
countries = r"فنزو|كويت|اران|ابو ظبي|قطر|عراق|خليجي|الجزائر|ليبي|نيجيري|سعودي|روسي|امريك|برازيل|امارات"

In [48]:
#Let's add a few more functions as well
@labeling_function()
def discover_oil(x):
    return POS if re.search(oil, x.text) and re.search("كشف|اكشاف", x.text) else ABSTAIN

companies = "شركه نفظ|سوناتراك|روسنفت|وطني للغاز|وطني للبترول"

@labeling_function()
def oil_companies(x):
    return POS if re.search(companies, x.text) else ABSTAIN

@labeling_function()
def economy_stupid(x):
    return POS if re.search(oil, x.text) and re.search(econ, x.text) else ABSTAIN 

In [49]:
lfs = [oil_terms, econ_terms, conflict_terms,
       misc_exclude, oil_countries, discover_oil,
      oil_companies, economy_stupid]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=train)
L_dev = applier.apply(df=dev)


100%|██████████| 3542/3542 [00:01<00:00, 2660.23it/s]

In [50]:
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
oil_terms,0,"[0, 1]",1.000000,1.000000,0.690853,129,48,0.036420
econ_terms,1,"[0, 1]",1.000000,1.000000,0.690853,143,34,0.040373
conflict_terms,2,[0],0.756352,0.756352,0.516657,0,168,0.000000
misc_exclude,3,[0],0.920949,0.920949,0.637493,0,169,0.000000
oil_countries,4,"[0, 1]",1.000000,1.000000,0.690853,158,19,0.044608
discover_oil,5,"[0, 1]",1.000000,1.000000,0.690853,12,165,0.003388
oil_companies,6,[0],1.000000,1.000000,0.690853,0,177,0.000000
economy_stupid,7,"[0, 1]",1.000000,1.000000,0.690853,105,72,0.029644


In [64]:
#Let's try to understand the functions that are always wrong
lfs = [conflict_terms,
       misc_exclude, oil_companies]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=train)
L_dev = applier.apply(df=dev)


100%|██████████| 3542/3542 [00:00<00:00, 6688.46it/s]

In [65]:
buckets = get_label_buckets(Y_dev, L_dev[:, 1])
dev.iloc[buckets[(POS, NEG)]].sample(5, random_state=1)

,text,category,oil,syria
6463,لافروف جزائر مقبل لبحث مكافحه ارهاب اسعار يبدا خارجيه روسي سيرغي لافروف مقبل زياره جزائر بدعوه نظيره رمطان لعمامره حسبما افاد بيان لوزاره شوون خارجيه جزائريه بيان وزيرين سيتطرقان باسهاب شراكه استراتيجيه تربط جزائر بروسيا بموجب اعلان موقع رئيسي بلدين سيتم زياره تقييم تعاون ثنائي مجالات اقتصاديه تقنيه علميه ثقافيه انعقاد دوره سابعه مشتركه جزائريه روسيه بموسكو ستتمحور محادثات جزائريه روسيه بيان مسائل اقليميه خاصه اوضاع ليبيا سوريا مكافحه ارهاب تطور نفطيه دوليه بيان كاله انبا جزائريه,أسواق_النفط,1,-1
6498,لقطات حصريه مناطق محرره تكشف اسرار تجاره داعشيه فريق تصوير قناه وثائقيه بجوله مناطق محرره داعش شمال سوريا شدادي رميلان قامشلي ابيض تمكن قرائن تكشف ابعاد ممارسات تنظيم داعشي يكشف تعاون تنظيم تركي فريق منطقه فتره اجرى عمليات تصوير مدينه شدادي قوات كرديه موخرا ايدي تنظيم داعش احكام سيطره شدادي مقاتلو حدات حمايه كرديه هائله وثائق دخول مقاتلين اجانب سوريا اظهر مقاتلو حدات حمايه كرديه جوازات وثائق هويه عثور ارهابيين قتلى وثائق جوازات جوازات داخليه لمواطنين فتاه هويات عراقيه جوازات كازاخيه ليبيه تونسيه بحرينيه اكبر وثائق ايديولوجي تنظيم مناهج دراسيه وثائق اختبارات لطلاب مدارس تابعه دوله اسلاميه كتاب يسمى عصابات قتال امثل لاسقاط نظام اسدي مجرم مواد صادره مكتبه غربا باسطنبول تتضمن وثائق معلومات فريده عناصر كتيبه يرموك فصيل شيشاني بتنظيم داعش بيانات شخصيه افراد اسلحه موجوده وثائق افراد كتيبه مرابطه فاروق يحصلون رواتب تراوحت دولارا قرابه دولارا شهريا متابعه تفاصيل قناه موقع فيلم وثائقي لاحق,أسواق_النفط,1,-1
21263,ايران تبرم اتفاقات نفط شركات عالميه لوك اويل قعت شركه نفط وطنيه ايرانيه اتفاقات اوليه شركات نفط اجنبيه لدراسه حقول نفط جمهوريه اسلاميه نقل موقع زاره نفط ايرانيه شانا رئيس شركه نفط وطنيه ايرانيه كاردور قوله عقود متعلقه بدراسه حقول نفط ايرانيه قعت شركات نمساويه توتال فرنسيه فينترشال المانيه برتامينا اندونيسيه لوك اويل زاروبج نفط روسيا تحتاج ايران استثمارات اجنبيه لاصلاح تحديث حقول نفط غاز تسعى لنقل تكنولوجيا قطا نفطي عزله استمرت نحو سنوات يزال شركات نفط غربيه آسيويه ينتظر اعلان طهران شروط عقود نفط غاز جديده,أسواق_النفط,1,-1
6400,اندلاع نيران خزانات بليبيا معارك داعش فيديو اندلعت نيران خزانات شمال ليبيا معارك حراس منشآت نفطيه مسلحي تنظيم داعش ارهابي اعلنت موسسه وطنيه ليبيه بيان نيران اندلعت خزانات معارك مينا سدره اكبر مينا لتخزين ليبيا متحدث باسم حراس منشآت نفطيه سدره فقدنا هجوم تنظيم دوله اسلاميه موانى يتلقى حراس دعما جويا قوات متمركزه قاعده مصراته مسوول سلاح قاعده واقعه طرابلس يسيطر قاعده تحالف ليبيا استولى طرابلس حكومه منافسه قائمه تاتي مواجهات تسعى متحده تشكيل حكومه طنيه يشهد فوضى تعترف اسره دوليه عاصمه طرابلس احتياطي نفطي افريقيا يقدر مليار برميل انتاجها مليون برميل يوميا تراجع,أسواق_النفط,1,-1
6656,دولي يساعد عراق مليار دولار صندوق دولي عراق بقيمه مليار دولار لمساعدته تخطي ازمه ماليه يواج تدهور اسعار حربه ارهاب صندوق دولي يمنح قرضا عراق اوضح صندوق بيان حكومه عراقيه ستحصل اولي بقيمه مليون دولار يتعين تنفيذ سلسله اصلاحات اقتصاديه حصول بقيه مساعده لمده ثلاث سنوات بيان مدرا مساعدين صندوق قوله اقتصاد عراقي عانى بشده صدمه مزدوجه ناجمه هجمات تنظيم دوله اسلاميه انخفاض شديد اسعار اعرب صندوق بيانه قلقه خصوصا متاخرات كبيره متراكمه بغداد تكافح تزاماتها ماليه صندوق بغداد قرضا بقيمه مليار دولار لمساعد نهوض عامه بموجب برنامج اداه تمويل سريع تاثر اقتصاد عراق بشكل تدني اسعار بشكل يعتبر اكبر اوسط تكاليف يعرف بتنظيم دوله اسلاميه محظور روسيا استيلا تنظيم عديد حقول استخراج,أسواق_النفط,1,-1
